In [1]:
# Code to generate objective function Jacobian matrices from solution models for local minimization
# it also generates additional inequality constraints

# last update 25.01.22, NR
import numpy as np
import math as math
import cmath

from sympy import *
from sympy.printing import print_ccode

# EXPORT
import os, re
init_printing()

In [2]:
# non linear phase solver, include option to generate ghost points
def func_bnd_guess(name,bulk):
    eps = 1e-14
    if   name == 'bi':
        # def bnd():
        t   = 1
        f   = 1

        if bulk[7] == 0:
            t  = eps
        if bulk[8] == 0:
            f  = eps

        box_bounds = ((0, 1), (0, 1),(0, 1), (0, t),(0, f))
        iguess = [0.15, 0.25, 0.4, 0.17, 0.25]
            
    elif name == 'ksp':
        # def bnd():
        box_bounds = ((0, 1), (0, 1))
        iguess = [0.1, 0.001]
        
    elif name == 'g':
        # def bnd():
        f    = 1
        cr   = 1
        t    = 1

        if bulk[8] == 0:
            f  = eps
        if bulk[9] == 0:
            cr  = eps
        if bulk[7] == 0:
            t  = eps

        box_bounds = ((0, 1), (0, 1),(0, f), (0, cr),(0, t))
        iguess = [0.3, 0.2, 0.01, 0.01, 0.001]

    elif name == 'ep':
        # def bnd():
        f   = 1
        Q   = 1

        if bulk[8] == 0:
            f  = eps
            Q  = eps
        box_bounds = ((0, f), (0, Q))
        iguess = [0.8, 0.03]
        
    elif name == 'pl':
        # def bnd():
        box_bounds = ((0, 1), (0, 1))
        iguess = [0.8, 0.03]
       
        
    elif name == 'mu':
        # def bnd():
        f   = 1

        if bulk[8] == 0:
            f  = eps
            
        box_bounds = ((0, 1), (0, 1),(0, f), (0, 1),(0, 1))
        iguess = [0.25, 0.6, 0.17, 0.06, 0.004]
       
    elif name == 'ol':
        # def bnd():
        box_bounds = ((0, 1), (0, 1),(0, 1))
        iguess = [0.4, 0.002, 0.01]
            
    elif name == 'cd':
        # def bnd():
        h   = 1

        if bulk[10] == 0:
            h  = eps
        box_bounds = ((0, 1), (0, h))
        iguess = [0.8, 0.03]

    elif name == 'opx':
        # def bnd():
        f    = 1
        cr   = 1
        t    = 1

        if bulk[8] == 0:
            f  = eps
        if bulk[9] == 0:
            cr  = eps
        if bulk[7] == 0:
            t  = eps

        box_bounds = ((0, 1), (0, 2),(0, 1), (-1, 1),(0, f),(0, t), (0, cr),(0, 1))
        iguess = [0.05, 0.006, 0.025, 0.032, 0.1, 0.001, 0.001, 0.001] 

            
    elif name == 'cpx':
        # def bnd():
        f    = 1
        cr   = 1
        t    = 1

        if bulk[8] == 0:
            f  = eps
        if bulk[9] == 0:
            cr  = eps
        if bulk[7] == 0:
            t  = eps

        box_bounds = ((0, 1), (0, 2),(0, 1), (0, 1),(-1, 1),(0, f), (0, cr),(0, t), (0, 1))
        iguess = [0.075, 0.1120, 0.05, 0.11, -0.0005, 0.001, 0.001, 0.001, 0.001]

            
    elif name == 'spn':
        # def bnd():
        y   = 1
        c   = 1
        t   = 1

        if bulk[7] == 0:
            t  = eps
        if bulk[8] == 0:
            y  = eps
        if bulk[9] == 0:
            c  = eps_
        box_bounds = ((0, 1), (0, y),(0, c), (0, t),(-1, 1),(-1, 1), (-1, 1))
        iguess = [0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.02]
            
    elif name == 'hb':
        # def bnd():
        f  = 1
        t  = 1

        if bulk[7] == 0:
            t  = eps
        if bulk[8] == 0:
            f  = eps

        box_bounds = ((0, 1), (0, 1),(0, 1), (0, 1),(0, 1),(0, 1), (0, f),(0, t), (-1, 1),(-1, 1))
        iguess = [0.3, 0.2, 0.01, 0.45, 0.01, 0.8, 0.05, 0.01, -0.01, 0.1]
            
    elif name == 'liq':
        # def bnd():
        hm  = 1
        ek  = 1
        ti  = 1
        h2o = 1
        
        if bulk[7] == 0:#ti
            ti   = eps
        if bulk[8] == 0:#chr
            hm  = eps
        if bulk[9] == 0:#o
            ek  = eps
        if bulk[10] == 0:#h2o
            h2o = eps

        box_bounds = ((0, 1), (0, 1),(0, 1), (0, 1),(0, 1), (0, hm),(0, ek), (0, ti),(0, 1), (0, 1), (0, h2o))
        iguess = [0.2,0.2,0.1,0.1,0.05,0.001,0.001,0.001,0.001,0.001,0.001]

    elif name == 'ilm':
        # def bnd():
        x2  = 0
        if bulk[8] == 0:
            x2  = 1 - eps

        box_bounds = ((x2, 1.), (-0.99, 0.99))
        iguess = [0.8, 0.55]

    else:
        display('Either phase is not defined or there is a typo')
  

    return (box_bounds,iguess)

def brackets(eq):  
    eq =  re.sub(r'[_][L][_]', r'[',str(eq) )
    eq =  re.sub(r'[_][R][_]', r']',str(eq) )
    return(eq) 

def digit2index(eq):  
    eq =  re.sub(r'[m][u][z]([0-9][0-9])', r'mu[\1]',str(eq) )
    eq =  re.sub(r'[m][u][_][G][e][x]([0-9][0-9])', r'mu_Gex[\1]',str(eq) )
    eq =  re.sub(r'[p][h][i]([0-9][0-9])', r'phi[\1]',str(eq) )
    eq =  re.sub(r'[i][d][m]([0-9][0-9])', r'idm[\1]',str(eq) )
    eq =  re.sub(r'[p]([0-9][0-9])', r'p[\1]',str(eq) )
    eq =  re.sub(r'[b][u][l][k]([0-9][0-9])', r'bulk[\1]',str(eq) )   
    eq =  re.sub(r'[c][o][m][p]([0-9][0-9])', r'chem_comp[\1]',str(eq) )   
    
    eq =  re.sub(r'[m][u][z]([0-9])', r'mu[\1]',str(eq) )  
    eq =  re.sub(r'[m][u][_][G][e][x]([0-9])', r'mu_Gex[\1]',str(eq) )
    eq =  re.sub(r'[p][h][i]([0-9])', r'phi[\1]',str(eq) )
    eq =  re.sub(r'[i][d][m]([0-9])', r'idm[\1]',str(eq) )
    eq =  re.sub(r'[p]([0-9])', r'p[\1]',str(eq) )
    eq =  re.sub(r'[c][o][m][p]([0-9])', r'chem_comp[\1]',str(eq) )
    eq =  re.sub(r'[b][u][l][k]([0-9])', r'bulk[\1]',str(eq) )  
    eq =  re.sub(r'[x]', r'x1',str(eq) ) #change x-eos name are already used by other variables
    eq =  re.sub(r'[n]', r'n1',str(eq) ) #change x-eos name are already used by other variables
    
    eq =  re.sub(r'[l][o][g]', r'fct_log',str(eq) ) # use fct_log, which deals with < 0 and NaN
    return(eq) 

def digit2index2(eq):  
    eq =  re.sub(r'[m][u][z]([0-9][0-9])', r'mu[\1]',str(eq) )
    eq =  re.sub(r'[m][u][_][G][e][x]([0-9][0-9])', r'mu_Gex[\1]',str(eq) )
    eq =  re.sub(r'[p][h][i]([0-9][0-9])', r'phi[\1]',str(eq) )
    eq =  re.sub(r'[i][d][m]([0-9][0-9])', r'idm[\1]',str(eq) )
    eq =  re.sub(r'[g][b]([0-9][0-9])', r'gb[\1]',str(eq) )
    eq =  re.sub(r'[p]([0-9][0-9])', r'p[\1]',str(eq) )
    eq =  re.sub(r'[b][u][l][k]([0-9][0-9])', r'bulk[\1]',str(eq) )   
    eq =  re.sub(r'[c][o][m][p]([0-9][0-9])', r'chem_comp[\1]',str(eq) )   
    
    eq =  re.sub(r'[m][u][z]([0-9])', r'mu[\1]',str(eq) )  
    eq =  re.sub(r'[m][u][_][G][e][x]([0-9])', r'mu_Gex[\1]',str(eq) )
    eq =  re.sub(r'[p][h][i]([0-9])', r'phi[\1]',str(eq) )
    eq =  re.sub(r'[i][d][m]([0-9])', r'idm[\1]',str(eq) )
    eq =  re.sub(r'[g][b]([0-9])', r'gb[\1]',str(eq) )
    eq =  re.sub(r'[p]([0-9])', r'p[\1]',str(eq) )
    eq =  re.sub(r'[c][o][m][p]([0-9])', r'chem_comp[\1]',str(eq) )
    eq =  re.sub(r'[b][u][l][k]([0-9])', r'bulk[\1]',str(eq) )  
#     eq =  re.sub(r'[x]', r'x1',str(eq) ) #change x-eos name are already used by other variables
#     eq =  re.sub(r'[n]', r'n1',str(eq) ) #change x-eos name are already used by other variables
    
    eq =  re.sub(r'[l][o][g]', r'fct_log',str(eq) ) # use fct_log, which deals with < 0 and NaN
    return(eq) 

def replaceSymbols(op,in_var,out_var):    
    regin  = 'r\'([' + in_var   +']*)\''
    regout = 'r\''  + out_var  +'\''
    t0 =  re.sub(in_var, out_var,op )  
    return(t0)

def realSquareRoot(eq):       
#     t1 =  re.sub(r'\[l][o][g](\([^()]*\))', r'(fct_log(\1))',str(eq) )
# WORKS (needs eps in the squares root)

    t1 =  re.sub(r'[l][o][g]', r'fct_log',str(eq) )
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.5', r'creal(pow(\1 + eps,0.5))',str(t1) )
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.25', r'creal(pow(\1 + eps,0.25))',str(t1) )
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.5\)', r'creal(pow(\1 + eps, -0.5))',str(t1) )  
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.25\)', r'creal(pow(\1 + eps, -0.25))',str(t1) ) 
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(0\.5\)', r'creal(pow(\1 + eps, 0.5))',str(t1) )  
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(0\.25\)', r'creal(pow(\1 + eps, 0.25))',str(t1) ) 
    
#     t1 =  re.sub(r'([^*]*)\*{2}0\.25', r'pow(\1 + eps, 0.25)',str(t1) ) 
#     t1 =  re.sub(r'([^*]*)\*{2}0\.5', r'pow(\1 + eps, 0.5)',str(t1) ) 
    
    
    
#     t1 =  re.sub(r'[l][o][g]', r'fct_log',str(eq) )
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.5', r'(creal(fsq(\1 +eps)**0.5))',str(t1) )
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.25', r'(creal(fsq(\1 +eps)**0.25))',str(t1) )
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.5\)', r'(creal(fnsq(\1 +eps)))',str(t1) )  
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.25\)', r'(creal(fnsq2(\1 +eps)))',str(t1) ) 

#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.5', r'(re(fsq(\1 )**0.5))',str(t1) )
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.25', r'(re(fsq(\1 )**0.25))',str(t1) )
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.5\)', r'(re(fnsq(\1 )))',str(t1) )  
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.25\)', r'(re(fnsq2(\1 )))',str(t1) )   

    return(t1) 

def realSquareRoot2(eq):       
#     t1 =  re.sub(r'\[l][o][g](\([^()]*\))', r'(fct_log(\1))',str(eq) )
# WORKS (needs eps in the squares root)

    t1 =  re.sub(r'[l][o][g]', r'fct_log',str(eq) )
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.5', r'pow(\1,0.5)',str(t1) )
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.25', r'pow(\1,0.25)',str(t1) )
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.5\)', r'pow(\1, -0.5)',str(t1) )  
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.25\)', r'pow(\1, -0.25)',str(t1) ) 
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(0\.5\)', r'pow(\1, 0.5)',str(t1) )  
    t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(0\.25\)', r'pow(\1, 0.25)',str(t1) ) 
    
    t1 =  re.sub(r'([^*]*)\*{2}0\.25', r'pow(\1, 0.25)',str(t1) ) 
    t1 =  re.sub(r'([^*]*)\*{2}0\.5', r'pow(\1, 0.5)',str(t1) ) 
    
    
    
#     t1 =  re.sub(r'[l][o][g]', r'fct_log',str(eq) )
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.5', r'(creal(fsq(\1 +eps)**0.5))',str(t1) )
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.25', r'(creal(fsq(\1 +eps)**0.25))',str(t1) )
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.5\)', r'(creal(fnsq(\1 +eps)))',str(t1) )  
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.25\)', r'(creal(fnsq2(\1 +eps)))',str(t1) ) 

#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.5', r'(re(fsq(\1 )**0.5))',str(t1) )
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}0\.25', r'(re(fsq(\1 )**0.25))',str(t1) )
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.5\)', r'(re(fnsq(\1 )))',str(t1) )  
#     t1 =  re.sub(r'\(([^)(]*)\)\*{2}\(\-0\.25\)', r'(re(fnsq2(\1 )))',str(t1) )   

    return(t1) 

def func_print(data):
    txt ='{'
    for i,val in enumerate(data):
        if i == len(data)-1:
            txt += str(val)
        else:
            txt += str(val)+', ' 
    
    txt +='}'    
    return(txt)

def func_print2(data):
    txt ='{'
    for i,val in enumerate(data):
        if i == len(data)-1:
            if len(val) > 1:
                txt +='{'
                for j,val2 in enumerate(val):
                    if j == len(val)-1:
                        txt += str(val2)
                    else:
                        txt += str(val2)+', ' 
                txt +='}'
            else:
                txt += str(val)
        else:
            if len(val) > 1:
                txt +='{'
                for j,val2 in enumerate(val):
                    if j == len(val)-1:
                        txt += str(val2)
                    else:
                        txt += str(val2)+', ' 
                txt +='},'
            else:
                txt += str(val)+', ' 
    
    txt +='}'    
    return(txt)

op = ''

In [ ]:
# ss database

def liq():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 12

    W         = [0]*66
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Initial guess -> ca,k
    IG       = [0]*(n_em-1);

    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # wo  range
    Range[1]  = [0,1]; # sl  range
    Range[2]  = [0,1]; # fo  range
    Range[3]  = [0,1]; # fa  range
    Range[4]  = [0,1]; # jd  range
    Range[5]  = [0,1]; # hm  range
    Range[6]  = [0,1]; # ek  range
    Range[7]  = [0,1]; # ti  range
    Range[8]  = [0,1]; # kj  range
    Range[9]  = [0,1]; # yct range
    Range[10] = [0,1]; # h2o range

    p[0] = sympify('1 - wo - sl - jd - fa - fo - hm - ek - ti - kj - h2o + 1/4*yct*(4 - 3*ek - 3*fa - 3*fo - 3*hm - 3*jd - 3*kj - 3*sl - 3*ti - 3*wo - 3*h2o)');
    p[1] = sympify('sl + 3/4*yct*sl + (-yct)');
    p[2] = sympify('wo + 3/4*yct*wo + (-yct)');
    p[3] = sympify('fo + 3/4*yct*fo');
    p[4] = sympify('fa + 3/4*yct*fa');
    p[5] = sympify('jd + 3/4*yct*jd');
    p[6] = sympify('hm + 3/4*yct*hm');
    p[7] = sympify('ek + 3/4*yct*ek');
    p[8] = sympify('ti + 3/4*yct*ti');
    p[9] = sympify('kj + 3/4*yct*kj');
    p[10] = sympify('yct');
    p[11] = sympify('h2o + 3/4*yct*h2o');

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 9.5 - 0.10*P;
    W[1] = -10.3;
    W[2] = -26.5 - 3.12*P;
    W[3] = -12.0 - 0.55*P;
    W[4] = -15.1 - 0.13*P;
    W[5] = 20.0;
    W[6] = 0;
    W[7] = 24.6;
    W[8] = -17.8 - 0.05*P;
    W[9] = -14.6;
    W[10] = 17.8 - 0.61*P;
    W[11] = -26.5 + 0.85*P;
    W[12] = 2.2;
    W[13] = 2.5;
    W[14] = 16.8;
    W[15] = -5.0;
    W[16] = 0;
    W[17] = 15.2 - 0.04*P;
    W[18] = 7.0;
    W[19] = 4.0;
    W[20] = 23.7 - 0.94*P;
    W[21] = 25.5 + 0.11*P;
    W[22] = 14.0;
    W[23] = -1.2;
    W[24] = 0;
    W[25] = 0;
    W[26] = 18.0;
    W[27] = -1.1;
    W[28] = 9.5;
    W[29] = 40.3 - 0.86*P;
    W[30] = 18.0;
    W[31] = 1.5;
    W[32] = 0;
    W[33] = 0;
    W[34] = 7.5;
    W[35] = 3.0;
    W[36] = -5.6;
    W[37] = 9.4 - 1.58*P;
    W[38] = 7.5 - 0.05*P;
    W[39] = -30.0;
    W[40] = 0;
    W[41] = 6.7;
    W[42] = 10.0;
    W[43] = -6.5;
    W[44] = 9.2 - 1.58*P;
    W[45] = 10.0;
    W[46] = 0;
    W[47] = 16.5 + 0.14*P;
    W[48] = -5.9;
    W[49] = 7.6;
    W[50] = -8.3 - 0.06*P; 
    W[51] = 0;
    W[52] = 0;
    W[53] = 10.0;
    W[54] = 0;
    W[55] = 60.0 - 0.66*P;
    W[56] = 0;
    W[57] = 0;
    W[58] = 0;
    W[59] = 30.0 - 0.66*P; 
    W[60] = 9.0;
    W[61] = 0;
    W[62] = 30.0 - 0.60*P;
    W[63] = -5.6;
    W[64] = -0.1 + 0.22*P;
    W[65] = 17.3 + 0.05*P;

    v[0] = 100.00;
    v[1] = 120.00;
    v[2] = 140.00;
    v[3] = 240.00;
    v[4] = 100.00;
    v[5] = 120.00;
    v[6] = 100.00;
    v[7] = 100.00;
    v[8] = 100.00;
    v[9] = 100.00;
    v[10] = 100.00;
    v[11] = 100.00;

    #     Site fractions
    pq   = sympify('1 - wo - sl - jd - fa - fo - hm - ek - ti - kj - h2o + 1/4*yct*(4 - 3*ek - 3*fa - 3*fo - 3*hm - 3*jd - 3*kj - 3*sl - 3*ti - 3*wo - 3*h2o)');
    psl  = sympify('sl + 3/4*yct*sl + (-yct)');
    pwo  = sympify('wo + 3/4*yct*wo + (-yct)');
    pjd  = sympify('jd + 3/4*yct*jd');
    phm  = sympify('hm + 3/4*yct*hm');
    pek  = sympify('ek + 3/4*yct*ek');
    pti  = sympify('ti + 3/4*yct*ti');
    pkj  = sympify('kj + 3/4*yct*kj');
    pct  = sympify('yct');
    pol  = sympify('fo + fa + 3/4*yct*(fo + fa)');
    sumT = sympify('1 - h2o + (-3/4*yct)*h2o');
    mgM  = sympify('4*fo');
    feM  = sympify('4*fa');
    CaM  = sympify('wo');
    AlM  = sympify('sl');
    sumM = sympify('4*fo + 4*fa + sl + wo');
    xh   = sympify('h2o');
    xv   = sympify('1 - h2o');

    sf = [pq, psl, pwo, pjd, phm, pek, pti, pkj, pct, pol, sumT, mgM, feM, CaM, AlM, sumM, xh, xv];

    pq   = sympify('sf_L_0_R_');
    psl  = sympify('sf_L_1_R_');
    pwo  = sympify('sf_L_2_R_');
    pjd  = sympify('sf_L_3_R_');
    phm  = sympify('sf_L_4_R_');
    pek  = sympify('sf_L_5_R_');
    pti  = sympify('sf_L_6_R_');
    pkj  = sympify('sf_L_7_R_');
    pct  = sympify('sf_L_8_R_');
    pol  = sympify('sf_L_9_R_');
    sumT = sympify('sf_L_10_R_');
    mgM  = sympify('sf_L_11_R_');
    feM  = sympify('sf_L_12_R_');
    CaM  = sympify('sf_L_13_R_');
    AlM  = sympify('sf_L_14_R_');
    sumM = sympify('sf_L_15_R_');
    xh   = sympify('sf_L_16_R_');
    xv   = sympify('sf_L_17_R_');
    
    idm[0]  =   xv**2*(sumT**-1)*pq;
    idm[1]  =   xv**2*(sumT**-1)*psl*AlM*sumM**-1;
    idm[2]  =   xv**2*(sumT**-1)*pwo*CaM*sumM**-1;
    idm[3]  =   xv**2*(sumT**-1)*pol*mgM**4*sumM**-4;
    idm[4]  =   xv**2*(sumT**-1)*pol*feM**4*sumM**-4;
    idm[5]  =   xv**2*(sumT**-1)*pjd;
    idm[6]  =   xv**2*(sumT**-1)*phm;
    idm[7]  =   xv**2*(sumT**-1)*pek;
    idm[8]  =   xv**2*(sumT**-1)*pti;
    idm[9]  =   xv**2*(sumT**-1)*pkj;
    idm[10] =   xv**2*(sumT**-1)*pct;
    idm[11] =   xh**2;



    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');
    gbase[3]  = sympify('gb4');
    gbase[4]  = sympify('gb5');
    gbase[5]  = sympify('gb6');
    gbase[6]  = sympify('gb7');
    gbase[7]  = sympify('gb8');
    gbase[8]  = sympify('gb9');
    gbase[9]  = sympify('gb10');
    gbase[10] = sympify('gb11');
    gbase[11] = sympify('gb12');


    symb     = ['wo','sl','fo','fa','jd','hm','ek','ti','kj','yct','h2o']
    sym_list = symb
    
    in_var  = ['wo','sl','fo','fa','jd','hm','ek','ti','kj','yct','h2o','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','x[7]','x[8]','x[9]','x[10]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def fl():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 11

    W         = [0]*55
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # wo  range
    Range[1]  = [0,1]; # sl  range
    Range[2]  = [0,1]; # fo  range
    Range[3]  = [0,1]; # fa  range
    Range[4]  = [0,1]; # jd  range
    Range[5]  = [0,1]; # hm  range
    Range[6]  = [0,1]; # ek  range
    Range[7]  = [0,1]; # ti  range
    Range[8]  = [0,1]; # kj  range
    Range[9]  = [0,1]; # h2o range
    


    p[0] = sympify('1 - wo - sl - fo - fa - jd - hm - ek - ti - kj - h2o');
    p[1] = sympify('sl');
    p[2] = sympify('wo');
    p[3] = sympify('fo');
    p[4] = sympify('fa');
    p[5] = sympify('jd');
    p[6] = sympify('hm');
    p[7] = sympify('ek');
    p[8] = sympify('ti');
    p[9] = sympify('kj');
    p[10] = sympify('h2o');

    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 0;
    W[1] = 0;
    W[2] = 0;
    W[3] = 0;
    W[4] = 0;
    W[5] = 0;
    W[6] = 0;
    W[7] = 0;
    W[8] = 0;
    W[9] = 59.0 - 0.82*P;
    W[10] = 0;
    W[11] = 0;
    W[12] = 0;
    W[13] = 0;
    W[14] = 0;
    W[15] = 0;
    W[16] = 0;
    W[17] = 0;
    W[18] = 57.6 - 0.80*P;
    W[19] = 0;
    W[20] = 0;
    W[21] = 0;
    W[22] = 0;
    W[23] = 0;
    W[24] = 0;
    W[25] = 0;
    W[26] = 72.2 - 0.67*P;
    W[27] = 0;
    W[28] = 0;
    W[29] = 0;
    W[30] = 0;
    W[31] = 0;
    W[32] = 0;
    W[33] = 71.7 - 1.10*P;
    W[34] = 0;
    W[35] = 0;
    W[36] = 0;
    W[37] = 0;
    W[38] = 0;
    W[39] = 71.7 - 1.10*P;
    W[40] = 0;
    W[41] = 0;
    W[42] = 0;
    W[43] = 0;
    W[44] = 57.0 - 0.79*P;
    W[45] = 0;
    W[46] = 0;
    W[47] = 0;
    W[48] = 73.0 - 0.66*P;
    W[49] = 0;
    W[50] = 0;
    W[51] = 73.0 - 0.66*P;
    W[52] = 0;
    W[53] = 75.0 - 0.67*P;
    W[54] = 44.9 - 1.19*P;

    v = 0;

    #     Site fractions
    pq  = sympify('1 - wo - sl - fo - fa - jd - hm - ek - ti - kj - h2o');
    psl = sympify('sl');
    pwo = sympify('wo');
    pfo = sympify('fo');
    pfa = sympify('fa');
    pjd = sympify('jd');
    phm = sympify('hm');
    pek = sympify('ek');
    pti = sympify('ti');
    pkj = sympify('kj');
    ph2o= sympify('h2o');
    fac = sympify('1 - h2o');

    sf = [pq, psl, pwo, pfo, pfa, pjd, phm, pek, pti, pkj, ph2o, fac];

    pq  = sympify('sf_L_0_R_');
    psl = sympify('sf_L_1_R_');
    pwo = sympify('sf_L_2_R_');
    pfo = sympify('sf_L_3_R_');
    pfa = sympify('sf_L_4_R_');
    pjd = sympify('sf_L_5_R_');
    phm = sympify('sf_L_6_R_');
    pek = sympify('sf_L_7_R_');
    pti = sympify('sf_L_8_R_');
    pkj = sympify('sf_L_9_R_');
    ph2o= sympify('sf_L_10_R_');
    fac = sympify('sf_L_11_R_'); 

    #     Ideal mixing parameters
    idm[0]  =   fac*pq;
    idm[1]  =   fac*psl;
    idm[2]  =   fac*pwo;
    idm[3]  =   fac*pfo;
    idm[4]  =   fac*pfa;
    idm[5]  =   fac*pjd;
    idm[6]  =   fac*phm;
    idm[7]  =   fac*pek;
    idm[8]  =   fac*pti;
    idm[9]  =   fac*pkj;
    idm[10] =  ph2o**2;
    

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');
    gbase[3]  = sympify('gb4');
    gbase[4]  = sympify('gb5');
    gbase[5]  = sympify('gb6');
    gbase[6]  = sympify('gb7');
    gbase[7]  = sympify('gb8');
    gbase[8]  = sympify('gb9');
    gbase[9]  = sympify('gb10');
    gbase[10] = sympify('gb11');


    symb     = ['wo','sl','fo','fa','jd','hm','ek','ti','kj','h2o']
    sym_list = symb;
    in_var  = ['wo','sl','fo','fa','jd','hm','ek','ti','kj','h2o','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','x[7]','x[8]','x[9]','x[10]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def pl4T():
    
    P        = symbols('P')
    T        = symbols('T')
    n_em     = 3 # 3 endmembers and 2 compositional variables

    W        = [0]*3
    v        = [0]*3
    gbase    = [0]*n_em;
    p        = [0]*n_em;
    idm      = [0]*n_em;
    idm2      = [0]*n_em;
    # Initial guess -> ca,k
    IG       = [0]*(n_em-1);
    IG[0]    = 0.8;
    IG[1]    = 0.03;
    
    # Validity bounds (need as many slack variables and Lagrange multiplier as there is range bounds)
    # Dim = n_emm - 1
    Range    = [[0]*2]*(n_em-1); # declare range
  
    Range[0] = [0,1]; # ca range
    Range[1] = [0,1]; # k range

    
    p[0]    = sympify('1 - k - ca');
    p[1]    = sympify('ca');
    p[2]    = sympify('k');

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0]    = 14.6 - 0.00935*T - 0.04*P;
    W[1]    = 24.1 - 0.00957*T + 0.338*P;
    W[2]    = 48.5 - 0.13*P;
    v[0]    = 0.674;
    v[1]    = 0.55;
    v[2]    = 1.0;

    #     Site fractions
    xNaA  = sympify('1 - ca - k');
    xCaA  = sympify('ca');
    xKA   = sympify('k');
    xAlTB = sympify('1/4 + 1/4*ca');
    xSiTB = sympify('3/4 - 1/4*ca');

    sf = [xNaA, xCaA, xKA, xAlTB, xSiTB];

    xNaA  = sympify('sf_L_0_R_');
    xCaA  = sympify('sf_L_1_R_');
    xKA   = sympify('sf_L_2_R_'); 
    xAlTB = sympify('sf_L_3_R_');
    xSiTB = sympify('sf_L_4_R_'); 
    
    #     Ideal mixing parameters
    idm[0] =   1.7548*xNaA*xAlTB**(1/4)*xSiTB**(3/4);
    idm[1] =   2*xCaA*xAlTB**(1/2)*xSiTB**(1/2);
    idm[2] =   1.7548*xKA*xAlTB**(1/4)*xSiTB**(3/4);

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');

    symb     = ['ca','k']
    sym_list = symb;
    
    in_var  = ['ca','k','log']   
    out_var = ['x[0]','x[1]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def ol():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 4

    W         = [0]*6
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [0,1]; # c range
    Range[2]  = [0,1]; # Q range

    p[0] = sympify('c');
    p[1] = sympify('-q + x');
    p[2] = sympify('1 - c - q - x + c*x');
    p[3] = sympify('2*q + (-c)*x');
    
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 24.;
    W[1] = 38.;
    W[2] = 24.;
    W[3] = 9.;
    W[4] = 4.5;
    W[5] = 4.5;    
    v    = 0.

    #     Site fractions
    xMgM1 = sympify('1 + q - x');
    xFeM1 = sympify('-q + x');
    xMgM2 = sympify('1 - c - q - x + c*x');
    xFeM2 = sympify('q + x + (-c)*x');
    xCaM2 = sympify('c');
    
    idm2[0] =   xMgM1*xCaM2;
    idm2[1] =   xFeM1*xFeM2;
    idm2[2] =   xMgM1*xMgM2;
    idm2[3] =   xMgM1*xFeM2;
    
    sf    = [xMgM1, xFeM1, xMgM2, xFeM2, xCaM2];
    
    xMgM1 = sympify('sf_L_0_R_');
    xFeM1 = sympify('sf_L_1_R_');
    xMgM2 = sympify('sf_L_2_R_');
    xFeM2 = sympify('sf_L_3_R_');
    xCaM2 = sympify('sf_L_4_R_');
    
    #     Ideal mixing parameters
    idm[0] =   xMgM1*xCaM2;
    idm[1] =   xFeM1*xFeM2;
    idm[2] =   xMgM1*xMgM2;
    idm[3] =   xMgM1*xFeM2;

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');
    gbase[3]  = sympify('gb4');
    
    symb     = ['x','c','q']
    sym_list = symb
    
    in_var  = ['x','c','q','log']   
    out_var = ['x[0]','x[1]','x[2]','ln']

    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def mu():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 6

    W         = [0]*15
    v         = [0]*6
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [0,1]; # y range
    Range[2]  = [0,1]; # f range
    Range[3]  = [0,1]; # n range
    Range[4]  = [0,1]; # c range

    
    p[0] = sympify('-c - f - n + y');
    p[1] = sympify('1. - x - y + x*y');
    p[2] = sympify('x + (-x)*y');
    p[3] = sympify('n');
    p[4] = sympify('c');
    p[5] = sympify('f');

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 0. + 0.20*P;
    W[1] = 0. + 0.20*P;
    W[2] = 10.12 + 0.0034*T + 0.353*P;
    W[3] = 35.0;
    W[4] = 0.;
    W[5] = 0.;
    W[6] = 45.0 + 0.25*P;
    W[7] = 50.0;
    W[8] = 0.;
    W[9] = 45.0 + 0.25*P;
    W[10] = 50.0;
    W[11] = 0.;
    W[12] = 15.0;
    W[13] = 30.0;
    W[14] = 35.0;
    v[0] = 0.63;
    v[1] = 0.63;
    v[2] = 0.63;
    v[3] = 0.37;
    v[4] = 0.63;
    v[5] = 0.63;

    #     Site fractions
    xKA     = sympify('1. - c - n');
    xNaA    = sympify('n');
    xCaA    = sympify('c');
    xMgM2A  = sympify('1. - x - y + x*y');
    xFeM2A  = sympify('x + (-x)*y');
    xAlM2A  = sympify('y');
    xAlM2B  = sympify('1. - f');
    xFe3M2B = sympify('f');
    xSiT1   = sympify('1. - 1./2.*c - 1./2.*y');
    xAlT1   = sympify('1./2.*c + 1./2.*y'); 
    
    sf      = [xKA, xNaA, xCaA, xMgM2A, xFeM2A, xAlM2A, xAlM2B, xFe3M2B, xSiT1, xAlT1];

    xKA     = sympify('sf_L_0_R_');
    xNaA    = sympify('sf_L_1_R_');
    xCaA    = sympify('sf_L_2_R_');
    xMgM2A  = sympify('sf_L_3_R_');
    xFeM2A  = sympify('sf_L_4_R_');
    xAlM2A  = sympify('sf_L_5_R_');
    xAlM2B  = sympify('sf_L_6_R_');
    xFe3M2B = sympify('sf_L_7_R_');
    xSiT1   = sympify('sf_L_8_R_');
    xAlT1   = sympify('sf_L_9_R_');
    
    #     Ideal mixing parameters
    idm[0] = 4*xKA*xAlM2A*xAlM2B*xSiT1*xAlT1;
    idm[1] =   xKA*xMgM2A*xAlM2B*(xSiT1**2);
    idm[2] =   xKA*xFeM2A*xAlM2B*(xSiT1**2);
    idm[3] = 4*xNaA*xAlM2A*xAlM2B*xSiT1*xAlT1;
    idm[4] =   xCaA*xAlM2A*xAlM2B*(xAlT1**2);
    idm[5] = 4*xKA*xAlM2A*xFe3M2B*xSiT1*xAlT1;

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');
    gbase[3]  = sympify('gb4');
    gbase[4]  = sympify('gb5');
    gbase[5]  = sympify('gb6');

    symb     = ['x','y','f','n','c']
    sym_list = symb
    
    in_var  = ['x','y','f','n','c','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def bi():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 6

    W         = [0]*15
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [0,1]; # y range
    Range[2]  = [0,1]; # f range
    Range[3]  = [0,1]; # t range
    Range[4]  = [0,1]; # Q range
    
    p[0] = sympify('1.0 - f - t - x - y - 2.0/3.0*q + f*x + t*x + x*y');
    p[1] = sympify('-1/3*q + x');
    p[2] = sympify('q -f*x -t*x -x*y');
    p[3] = sympify('y');
    p[4] = sympify('t');
    p[5] = sympify('f');

    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 12.;
    W[1] = 4.;
    W[2] = 10.;
    W[3] = 30.;
    W[4] = 8.;
    W[5] = 8.;
    W[6] = 5.;
    W[7] = 32.;
    W[8] = 13.6;
    W[9] = 7.;
    W[10] = 24.;
    W[11] = 5.6;
    W[12] = 40.0;
    W[13] = 1.0;
    W[14] = 40.0;
    v = 0.;

    #     Site fractions
    xMgM3 = sympify('1 - f - t - x - y - 2/3*q + f*x + t*x + x*y');
    xFeM3 = sympify('x + 2/3*q + (-f)*x + (-t)*x + (-x)*y');
    xFe3M3 = sympify('f');
    xTiM3 = sympify('t');
    xAlM3 = sympify('y');
    xMgM12 = sympify('1 + 1/3*q - x');
    xFeM12 = sympify('-1/3*q + x');
    xSiT = sympify('1/2 - 1/2*f - 1/2*y');
    xAlT = sympify('1/2 + 1/2*f + 1/2*y');
    xOHV = sympify('1 - t');
    xOV = sympify('t');
    
    sf  = [xMgM3, xFeM3, xFe3M3, xTiM3, xAlM3, xMgM12, xFeM12, xSiT, xAlT, xOHV, xOV];
     
    xMgM3  = sympify('sf_L_0_R_');
    xFeM3  = sympify('sf_L_1_R_');
    xFe3M3 = sympify('sf_L_2_R_');
    xTiM3  = sympify('sf_L_3_R_');
    xAlM3  = sympify('sf_L_4_R_');
    xMgM12 = sympify('sf_L_5_R_');
    xFeM12 = sympify('sf_L_6_R_');
    xSiT   = sympify('sf_L_7_R_');
    xAlT   = sympify('sf_L_8_R_');
    xOHV   = sympify('sf_L_9_R_');
    xOV    = sympify('sf_L_10_R_'); 
    
    #     Ideal mixing parameters
    idm[0] = 4*xMgM3*xMgM12**2*xSiT*xAlT*xOHV**2;
    idm[1] =   4*xFeM3*xFeM12**2*xSiT*xAlT*xOHV**2;
    idm[2] =   4*xFeM3*xMgM12**2*xSiT*xAlT*xOHV**2;
    idm[3] = xAlM3*xMgM12**2*xAlT**2*xOHV**2;
    idm[4] =   4*xTiM3*xMgM12**2*xSiT*xAlT*xOV**2;
    idm[5] = xFe3M3*xMgM12**2*xAlT**2*xOHV**2;

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');
    gbase[3]  = sympify('gb4');
    gbase[4]  = sympify('gb5');
    gbase[5]  = sympify('gb6');

    
    symb     = ['x','y','f','t','q']
    sym_list = symb
    
    in_var  = ['x','y','f','t','q','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def g():
    P = symbols('P')
    T = symbols('T')
    n_em = 6

    W         = [0]*15
    v         = [0]*6
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x  range
    Range[1]  = [0,1]; # c  range
    Range[2]  = [0,1]; # f  range
    Range[3]  = [0,1]; # cr range
    Range[4]  = [0,1]; # t  range

    p[0] = sympify('1 - c - cr - x - 4*t + c*x');
    p[1] = sympify('x + (-c)*x');
    p[2] = sympify('c - f');
    p[3] = sympify('f');
    p[4] = sympify('cr');
    p[5] = sympify('4*t');

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 4.0 + 0.10*P;
    W[1] = 45.4 - 0.010*T + 0.04*P;
    W[2] = 107.0 - 0.010*T - 0.036*P;
    W[3] = 2.0;
    W[4] = 0;
    W[5] = 17.0 - 0.010*T + 0.10*P;
    W[6] = 65.0 - 0.010*T + 0.039*P;
    W[7] = 6.0 + 0.01*P;
    W[8] = 0;
    W[9] = 2.0;
    W[10] = 1.0 - 0.010*T + 0.18*P;
    W[11] = 0;
    W[12] = 63.0 - 0.010*T + 0.10*P;
    W[13] = 0;
    W[14] = 0;
    v[0] = 1.00;
    v[1] = 1.00;
    v[2] = 2.50;
    v[3] = 2.50;
    v[4] = 1.00;
    v[5] = 1.00;

    #     Site fractions
    xMgM1  = sympify('1 - c - x + c*x');
    xFeM1  = sympify('x + (-c)*x');
    xCaM1  = sympify('c');
    xAlM2  = sympify('1 - cr - f - 2*t');
    xCrM2  = sympify('cr');
    xFe3M2 = sympify('f');
    xMgM2  = sympify('t');
    xTiM2  = sympify('t');

    sf    = [xMgM1, xFeM1, xCaM1, xAlM2, xCrM2, xFe3M2, xMgM2, xTiM2];      
    
    xMgM1  = sympify('sf_L_0_R_');
    xFeM1  = sympify('sf_L_1_R_');
    xCaM1  = sympify('sf_L_2_R_');
    xAlM2  = sympify('sf_L_3_R_');
    xCrM2  = sympify('sf_L_4_R_');
    xFe3M2 = sympify('sf_L_5_R_');
    xMgM2  = sympify('sf_L_6_R_');
    xTiM2  = sympify('sf_L_7_R_');
    p1     = sympify('sf_L_8_R_');
    
    #     Ideal mixing parameters
    idm[0] =   xMgM1**3*xAlM2**2;
    idm[1] =   xFeM1**3*xAlM2**2;
    idm[2] =   xCaM1**3*xAlM2**2;
    idm[3] =   xCaM1**3*xFe3M2**2;
    idm[4] =   xMgM1**3*xCrM2**2;
    idm[5] = 8*xMgM1**3*xAlM2*xMgM2**(1/2)*xTiM2**(1/2);

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');
    gbase[3]  = sympify('gb4');
    gbase[4]  = sympify('gb5');
    gbase[5]  = sympify('gb6');

    symb     = ['x','c','f','cr','t']
    sym_list = symb
    
    in_var  = ['x','cr','f','c','t','log']   
    out_var = ['x[0]','x[3]','x[2]','x[1]','x[4]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def ep():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 3

    W         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];   # f range
    Range[1]  = [0,0.5]; # Q range

    p[0] = sympify('1 - f - q');
    p[1] = sympify('2*q');
    p[2] = sympify('f - q');
    
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 1.;
    W[1] = 3.;
    W[2] = 1.;    
    v = 0.; 

    #     Site fractions
    xFeM1 = sympify('f - q');
    xAlM1 = sympify('1 - f + q');
    xFeM3 = sympify('f + q');
    xAlM3 = sympify('1 - f - q');

    
    sf    = [xFeM1, xAlM1, xFeM3, xAlM3];

    xFeM1 = sympify('sf_L_0_R_');
    xAlM1 = sympify('sf_L_1_R_');
    xFeM3 = sympify('sf_L_2_R_');
    xAlM3 = sympify('sf_L_3_R_');
    
    #     Ideal mixing parameters
    idm[0] =   xAlM1*xAlM3;
    idm[1] =   xAlM1*xFeM3;
    idm[2] =   xFeM1*xFeM3;
    
    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');

    
    symb     = ['f','q']
    sym_list = symb
    
    in_var  = ['f','q','log']   
    out_var = ['x[0]','x[1]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def cd():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 3

    W         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [0,1]; # h range

    
    p[0] = sympify('1 - h - x');
    p[1] = sympify('x');
    p[2] = sympify('h');
    
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 6.;
    W[1] = 0.;
    W[2] = 0.;    
    v = 0.; 

    #     Site fractions
    xFeX  = sympify('x');
    xMgX  = sympify('1 - x');
    xH2OH = sympify('h');
    xvH   = sympify('1 - h');

    sf  = [xFeX, xMgX, xH2OH, xvH];

    xFeX  = sympify('sf_L_0_R_');
    xMgX  = sympify('sf_L_1_R_');
    xH2OH = sympify('sf_L_2_R_');
    xvH   = sympify('sf_L_3_R_');   

    #     Ideal mixing parameters
    idm[0] =   xMgX**2*xvH;
    idm[1] =   xFeX**2*xvH;
    idm[2] =   xMgX**2*xH2OH;
    
    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');

    symb     = ['x','h']
    sym_list = symb
    
    in_var  = ['x','h','log']   
    out_var = ['x[0]','x[1]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def opx():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 9
    idm2 = []

    W         = [0]*36
    v         = [0]*9
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];  # x range
    Range[1]  = [0,2];  # y range
    Range[2]  = [0,1];  # c range         # Range(3,:) = [ 0 0.5], Boris correction commented
    Range[3]  = [-1,1]; # Q range
    Range[4]  = [0,1];  # f range
    Range[5]  = [0,1];  # t range
    Range[6]  = [0,1];  # cr range
    Range[7]  = [0,1];  # j range
    
    p[0] = sympify('1 - c - j + q - x - y + (-j)*q + q*t + c*x + j*x + (-q)*y');
    p[1] = sympify('q + x + (-j)*q + q*t + (-j)*x + t*x + (-q)*y + (-x)*y');
    p[2] = sympify('-2*q + 2*j*q + (-2*q)*t + (-c)*x + (-t)*x + 2*q*y + x*y');
    p[3] = sympify('c');
    p[4] = sympify('-cr - f + y - 2*t');
    p[5] = sympify('cr');
    p[6] = sympify('2*t');
    p[7] = sympify('f');
    p[8] = sympify('j');

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 7.0;
    W[1] = 4.0;
    W[2] = 29.4;
    W[3] = 12.5 - 0.04*P;
    W[4] = 8.0;
    W[5] = 6.0;
    W[6] = 8.0;
    W[7] = 35.0;
    W[8] = 4.0;
    W[9] = 21.5 + 0.08*P;
    W[10] = 11.0 - 0.15*P;
    W[11] = 10.0;
    W[12] = 7.0;
    W[13] = 10.0;
    W[14] = 35.0;
    W[15] = 18.0 + 0.08*P;
    W[16] = 15.0 - 0.15*P;
    W[17] = 12.0;
    W[18] = 8.0;
    W[19] = 12.0;
    W[20] = 35.0;
    W[21] = 75.5 - 0.84*P;
    W[22] = 20.0;
    W[23] = 40.0;
    W[24] = 20.0;
    W[25] = 35.0;
    W[26] = 2.0;
    W[27] = 10.0;
    W[28] = 2.0;
    W[29] = 7.0;
    W[30] = 6.0;
    W[31] = 2.0;
    W[32] = -11.0;
    W[33] = 6.0;
    W[34] = 20.0;
    W[35] = -11.0;
    v[0] = 1.00;
    v[1] = 1.00;
    v[2] = 1.00;
    v[3] = 1.20;
    v[4] = 1.00;
    v[5] = 1.00;
    v[6] = 1.00;
    v[7] = 1.00;
    v[8] = 1.20;

    #     Site fractions
    xMgM1  = sympify('1 - j - q + t - x - y + j*q + (-q)*t + j*x + (-t)*x + q*y + x*y');
    xFeM1  = sympify('q + x + (-j)*q + q*t + (-j)*x + t*x + (-q)*y + (-x)*y');
    xAlM1  = sympify('-cr - f + j + y - 2*t');
    xFe3M1 = sympify('f');
    xCrM1  = sympify('cr');
    xTiM1  = sympify('t');
    xMgM2  = sympify('1 - c - j + q - x + (-j)*q + q*t + c*x + j*x + (-q)*y');
    xFeM2  = sympify('-q + x + j*q + (-q)*t + (-c)*x + (-j)*x + q*y');
    xCaM2  = sympify('c');
    xNaM2  = sympify('j');
    xSiT   = sympify('1 - 1/2*y');
    xAlT   = sympify('1/2*y');

    sf   = [xMgM1, xFeM1, xAlM1, xFe3M1, xCrM1, xTiM1, xMgM2, xFeM2, xCaM2, xNaM2, xSiT, xAlT];

    xMgM1  = sympify('sf_L_0_R_');
    xFeM1  = sympify('sf_L_1_R_');
    xAlM1  = sympify('sf_L_2_R_');
    xFe3M1 = sympify('sf_L_3_R_');
    xCrM1  = sympify('sf_L_4_R_');
    xTiM1  = sympify('sf_L_5_R_');
    xMgM2  = sympify('sf_L_6_R_');
    xFeM2  = sympify('sf_L_7_R_');
    xCaM2  = sympify('sf_L_8_R_');
    xNaM2  = sympify('sf_L_9_R_');
    xSiT   = sympify('sf_L_10_R_');
    xAlT   = sympify('sf_L_11_R_');
    p1     = sympify('sf_L_12_R_');
    p5     = sympify('sf_L_13_R_');
    #     Ideal mixing parameters
    idm[0] =   xMgM1*xMgM2*xSiT**(1/2);
    idm[1] =   xFeM1*xFeM2*xSiT**(1/2);
    idm[2] =   xMgM1*xFeM2*xSiT**(1/2);
    idm[3] =   xMgM1*xCaM2*xSiT**(1/2);
    idm[4] = 1.4142*xAlM1*xMgM2*xSiT**(1/4)*xAlT**(1/4);
    idm[5] = 1.4142*xCrM1*xMgM2*xSiT**(1/4)*xAlT**(1/4);
    idm[6] = 2.8284*xMgM1**(1/2)*xTiM1**(1/2)*xMgM2*xSiT**(1/4)*xAlT**(1/4);
    idm[7] = 1.4142*xFe3M1*xMgM2*xSiT**(1/4)*xAlT**(1/4);
    idm[8] =   xAlM1*xNaM2*xSiT**(1/2);
    
    idm2[0] =   xMgM1*xMgM2*xSiT**(1/2)/p[0];
    idm2[1] =   xFeM1*xFeM2*xSiT**(1/2)/p[1];
    idm2[2] =   xMgM1*xFeM2*xSiT**(1/2)/p[2];
    idm2[3] =   xMgM1*xCaM2*xSiT**(1/2)/p[3];
    idm2[4] = 1.4142*xAlM1*xMgM2*xSiT**(1/4)*xAlT**(1/4)/p[4];
    idm2[5] = 1.4142*xCrM1*xMgM2*xSiT**(1/4)*xAlT**(1/4)/p[5];
    idm2[6] = 2.8284*xMgM1**(1/2)*xTiM1**(1/2)*xMgM2*xSiT**(1/4)*xAlT**(1/4)/p[6];
    idm2[7] = 1.4142*xFe3M1*xMgM2*xSiT**(1/4)*xAlT**(1/4)/p[7];
    idm2[8] =   xAlM1*xNaM2*xSiT**(1/2)/p[8];
    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');
    gbase[3]  = sympify('gb4');
    gbase[4]  = sympify('gb5');
    gbase[5]  = sympify('gb6');
    gbase[6]  = sympify('gb7');
    gbase[7]  = sympify('gb8');
    gbase[8]  = sympify('gb9');
    
    in_var  = ['x','y','cr','q','f','t','c','j','log']   
    out_var = ['x[0]','x[1]','x[6]','x[3]','x[4]','x[5]','x[2]','x[7]','ln']
    
    symb     = ['x','y','c','q','f','t','cr','j']
    sym_list = symb
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def cpx():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 10

    W         = [0]*45
    v         = [0]*10
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
# Validity bounds
    Range    = [[0]*2]*(n_em); # declare range
    
    Range[0]  = [0,1];  # x range
    Range[1]  = [0,2];  # y range
    Range[2]  = [0,1];  # o range
    Range[3]  = [-1,1]; # n range
    Range[4]  = [0,1];  # Q range
    Range[5]  = [0,1];  # f range
    Range[6]  = [0,1];  # cr range
    Range[7]  = [0,1];  # t range
    Range[8]  = [0,1];  # k range
    
    p[0] = sympify('1 - k - n - o - y');
    p[1] = sympify('q + x + (-k)*q + (-n)*q + q*t + (-k)*x + (-n)*x + t*x + (-q)*y + (-x)*y');
    p[2] = sympify('-cr - f + y - 2*t');
    p[3] = sympify('cr');
    p[4] = sympify('f');
    p[5] = sympify('2*t');
    p[6] = sympify('n');
    p[7] = sympify('o + q + (-k)*q + (-n)*q + q*t + (-o)*x + (-q)*y');
    p[8] = sympify('-x - 2*q + 2*k*q + 2*n*q + (-2*q)*t + k*x + n*x + o*x + (-t)*x + 2*q*y + x*y');
    p[9] = sympify('k');
    
    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 25.8;
    W[1] = 13.0 - 0.06*P;
    W[2] = 8.0;
    W[3] = 8.0;
    W[4] = 8.0;
    W[5] = 26.0;
    W[6] = 29.8;
    W[7] = 20.6;
    W[8] = 26.0;
    
    W[9] = 25.0 - 0.10*P;
    W[10] = 38.3;
    W[11] = 43.3;
    W[12] = 24.0;
    W[13] = 24.0;
    W[14] = 2.3;
    W[15] = 3.5;
    W[16] = 24.0;
    
    W[17] = 2.0;
    W[18] = 2.0;
    W[19] = 6.0;
    W[20] = 6.0;
    W[21] = 45.2 - 0.35*P;
    W[22] = 27.0 - 0.10*P;
    W[23] = 6.0;
    
    W[24] = 2.0;
    W[25] = 6.0;
    W[26] = 3.0;
    W[27] = 52.3;
    W[28] = 40.3;
    W[29] = 3.0;
    
    W[30] = 6.0;
    W[31] = 3.0;
    W[32] = 57.3;
    W[33] = 45.3;
    W[34] = 3.0;
    
    W[35] = 16.0;
    W[36] = 24.0;
    W[37] = 22.0;
    W[38] = 16.0;
    
    W[39] = 40.0;
    W[40] = 40.0;
    W[41] = 28.0;
    
    W[42] = 4.0;
    W[43] = 40.0;
    W[44] = 40.0;
    v[0] = 1.20;
    v[1] = 1.00;
    v[2] = 1.90;
    v[3] = 1.90;
    v[4] = 1.90;
    v[5] = 1.90;
    v[6] = 1.20;
    v[7] = 1.00;
    v[8] = 1.00;
    v[9] = 1.20;

    #     Site fractions
    xMgM1  = sympify('1 - k - n - q + t - x - y + k*q + n*q + (-q)*t + k*x + n*x + (-t)*x + q*y + x*y');
    xFeM1  = sympify('q + x + (-k)*q + (-n)*q + q*t + (-k)*x + (-n)*x + t*x + (-q)*y + (-x)*y');
    xAlM1  = sympify('-cr - f + k + n + y - 2*t');
    xFe3M1 = sympify('f');
    xCrM1  = sympify('cr');
    xTiM1  = sympify('t');
    xMgM2  = sympify('o + q + (-k)*q + (-n)*q + q*t + (-o)*x + (-q)*y');
    xFeM2  = sympify('-q + k*q + n*q + (-q)*t + o*x + q*y');
    xCaM2  = sympify('1 - k - n - o');
    xNaM2  = sympify('n');
    xKM2   = sympify('k');
    xSiT   = sympify('1 - 1/2*y');
    xAlT   = sympify('1/2*y');

    sf   = [xMgM1, xFeM1, xAlM1, xFe3M1, xCrM1, xTiM1, xMgM2, xFeM2, xCaM2, xNaM2, xKM2, xSiT, xAlT];
    
    xMgM1  = sympify('sf_L_0_R_');
    xFeM1  = sympify('sf_L_1_R_');
    xAlM1  = sympify('sf_L_2_R_');
    xFe3M1 = sympify('sf_L_3_R_');
    xCrM1  = sympify('sf_L_4_R_');
    xTiM1  = sympify('sf_L_5_R_');
    xMgM2  = sympify('sf_L_6_R_');
    xFeM2  = sympify('sf_L_7_R_');
    xCaM2  = sympify('sf_L_8_R_');
    xNaM2  = sympify('sf_L_9_R_');
    xKM2   = sympify('sf_L_10_R_');
    xSiT   = sympify('sf_L_11_R_');
    xAlT   = sympify('sf_L_12_R_');
    p3     = sympify('sf_L_13_R_');
    
    #     Ideal mixing parameters
    idm[0] =   xMgM1*xCaM2*xSiT**(1/2);
    idm[1] =   xFeM1*xFeM2*xSiT**(1/2);
    idm[2] = 1.4142*xAlM1*xCaM2*xSiT**(1/4)*xAlT**(1/4);
    idm[3] = 1.4142*xCrM1*xCaM2*xSiT**(1/4)*xAlT**(1/4);
    idm[4] = 1.4142*xFe3M1*xCaM2*xSiT**(1/4)*xAlT**(1/4);
    idm[5] = 2.8284*xMgM1**(1/2)*xTiM1**(1/2)*xCaM2*xSiT**(1/4)*xAlT**(1/4);
    idm[6] =   xAlM1*xNaM2*xSiT**(1/2);
    idm[7] =   xMgM1*xMgM2*xSiT**(1/2);
    idm[8] =   xMgM1*xFeM2*xSiT**(1/2);
    idm[9]=   xAlM1*xKM2*xSiT**(1/2);
 
    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');
    gbase[3]  = sympify('gb4');
    gbase[4]  = sympify('gb5');
    gbase[5]  = sympify('gb6');
    gbase[6]  = sympify('gb7');
    gbase[7]  = sympify('gb8');
    gbase[8]  = sympify('gb9');
    gbase[9]  = sympify('gb10');

    symb     = ['x','y','o','n','q','f','cr','t','k']
    sym_list = symb
    
    in_var  = ['x','y','o','n','q','f','cr','t','k','log']  
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','x[7]','x[8]','ln']
    
    
    return sym_list, p, v, W, gbase, sf, idm, idm2,  n_em, symmetry, Range, in_var, out_var

def hb():
    
    P = symbols('P')
    T = symbols('T')
    eps_hb = symbols('eps')
    n_em = 11

    W         = [0]*55
    v         = [0]*11
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];  # x range
    Range[1]  = [0,2];  # y range
    Range[2]  = [0,1];  # z range         # Range(3,:) = [ 0 0.5], Boris correction commented
    Range[3]  = [0,1];  # a range
    Range[4]  = [0,1];  # k range
    Range[5]  = [0,1];  # c range
    Range[6]  = [0,1];  # f range
    Range[7]  = [0,1];  # t range
    Range[8]  = [-1,1]; # Q1 range
    Range[9]  = [-1,1]; # Q2 range

    
    p[0] = sympify('-1/2*a + c - f - t - y + z');
    p[1] = sympify('-1/2*a + f + y - z');
    p[2] = sympify('a + (-a)*k');
    p[3] = sympify('-f + z');
    p[4] = sympify('1 - c - Q2 - x - z - 3/2*Q1 + f*Q2 + Q2*t + c*x + Q2*y + x*z');
    p[5] = sympify('x - 2*Q2 - 5/2*Q1 + 2*f*Q2 + 2*Q2*t + c*x + (-f)*x + (-t)*x + 2*Q2*y + (-x)*y + x*z');
    p[6] = sympify('Q2 + 5/2*Q1 + (-f)*Q2 + (-Q2)*t + (-c)*x + (-Q2)*y + (-x)*z');
    p[7] = sympify('2*Q2 + 3/2*Q1 + (-2*f)*Q2 + (-2*Q2)*t + (-c)*x + f*x + t*x + (-2*Q2)*y + x*y + (-x)*z');
    p[8] = sympify('f');
    p[9] = sympify('a*k');
    p[10] = sympify('t'); 

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 20.0;
    W[1] = 25.0;
    W[2] = 65.0;
    W[3] = 45.0;
    W[4] = 75.0;
    W[5] = 57.0;
    W[6] = 63.0;
    W[7] = 52.0;
    W[8] = 30.0;
    W[9] = 85.0;
    W[10] = -40.0;
    W[11] = 25.0;
    W[12] = 70.0;
    W[13] = 80.0;
    W[14] = 70.0;
    W[15] = 72.5;
    W[16] = 20.0;
    W[17] = -40.0;
    W[18] = 35.0;
    W[19] = 50.0;
    W[20] = 90.0;
    W[21] = 106.7;
    W[22] = 94.8;
    W[23] = 94.8;
    W[24] = 40.0;
    W[25] = 8.0;
    W[26] = 15.0;
    W[27] = 100.0;
    W[28] = 113.5;
    W[29] = 100.0;
    W[30] = 111.2;
    W[31] = 0.0;
    W[32] = 54.0;
    W[33] = 75.0;
    W[34] = 33.0;
    W[35] = 18.0;
    W[36] = 23.0;
    W[37] = 80.0;
    W[38] = 87.0;
    W[39] = 100.0;
    W[40] = 12.0;
    W[41] = 8.0;
    W[42] = 91.0;
    W[43] = 96.0;
    W[44] = 65.0;
    W[45] = 20.0;
    W[46] = 80.0;
    W[47] = 94.0;
    W[48] = 95.0;
    W[49] = 90.0;
    W[50] = 94.0;
    W[51] = 95.0;
    W[52] = 50.0;
    W[53] = 50.0;
    W[54] = 35.0;
    v[0] = 1.00;
    v[1] = 1.50;
    v[2] = 1.70;
    v[3] = 0.80;
    v[4] = 1.00;
    v[5] = 1.00;
    v[6] = 1.00;
    v[7] = 1.00;
    v[8] = 0.80;
    v[9] = 1.70;
    v[10] = 1.50;

    #     Site fractions
    xvA     = sympify('1 - a');
    xNaA    = sympify('a + (-a)*k');
    xKA     = sympify('a*k');
    xMgM13  = sympify('1 + Q1 - x');
    xFeM13  = sympify('-Q1 + x');
    xMgM2   = sympify('1 - f + Q2 - t - x - y + (-f)*Q2 + (-Q2)*t + f*x + t*x + (-Q2)*y + x*y');
    xFeM2   = sympify('-Q2 + x + f*Q2 + Q2*t + (-f)*x + (-t)*x + Q2*y + (-x)*y');
    xAlM2   = sympify('y');
    xFe3M2  = sympify('f');
    xTiM2   = sympify('t');
    xCaM4   = sympify('c');
    xMgM4   = sympify('1 - c - Q2 - x - z - 3/2*Q1 + f*Q2 + Q2 *t + c*x + Q2*y + x*z');
    xFeM4   = sympify('Q2 + x + 3/2*Q1 + (-f)*Q2 + (-Q2)*t + (-c)*x + (-Q2)*y + (-x)*z');
    xNaM4   = sympify('z');
    xSiT1   = sympify('1 - 1/2*f - 1/2*t - 1/2*y + 1/2*z - 1/4*a');
    xAlT1   = sympify('1/2*f + 1/2*t + 1/2*y - 1/2*z + 1/4*a');
    xOHV    = sympify('1 - t');
    xOV     = sympify('t');
    
    #     Ideal mixing parameters
    idm2[0] =   xvA*xMgM13**3*xMgM2**2*xCaM4**2*xSiT1*xOHV**2;
    idm2[1] = 2*xvA*xMgM13**3*xAlM2**2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOHV**2;
    idm2[2] = 8*xNaA*xMgM13**3*xMgM2*xAlM2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOHV**2;
    idm2[3] =   xvA*xMgM13**3*xAlM2**2*xNaM4**2*xSiT1*xOHV**2;
    idm2[4] =   xvA*xMgM13**3*xMgM2**2*xMgM4**2*xSiT1*xOHV**2;
    idm2[5] =   xvA*xFeM13**3*xFeM2**2*xFeM4**2*xSiT1*xOHV**2;
    idm2[6] =   xvA*xMgM13**3*xFeM2**2*xFeM4**2*xSiT1*xOHV**2;
    idm2[7] =   xvA*xFeM13**3*xMgM2**2*xFeM4**2*xSiT1*xOHV**2;
    idm2[8] =   xvA*xMgM13**3*xFe3M2**2*xNaM4**2*xSiT1*xOHV**2;
    idm2[9] = 8*xKA*xMgM13**3*xMgM2*xAlM2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOHV**2;
    idm2[10] = 2*xvA*xMgM13**3*xTiM2**2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOV**2;

    sf  = [xvA, xNaA, xKA, xMgM13, xFeM13, xMgM2, xFeM2, xAlM2, xFe3M2, xTiM2, xCaM4, xMgM4, xFeM4, xNaM4, xSiT1, xAlT1, xOHV, xOV];

    xvA     = sympify('sf_L_0_R_');
    xNaA    = sympify('sf_L_1_R_');
    xKA     = sympify('sf_L_2_R_');
    xMgM13  = sympify('sf_L_3_R_');
    xFeM13  = sympify('sf_L_4_R_');
    xMgM2   = sympify('sf_L_5_R_');
    xFeM2   = sympify('sf_L_6_R_');
    xAlM2   = sympify('sf_L_7_R_');
    xFe3M2  = sympify('sf_L_8_R_');
    xTiM2   = sympify('sf_L_9_R_');
    xCaM4   = sympify('sf_L_10_R_');
    xMgM4   = sympify('sf_L_11_R_');
    xFeM4   = sympify('sf_L_12_R_');
    xNaM4   = sympify('sf_L_13_R_');
    xSiT1   = sympify('sf_L_14_R_');
    xAlT1   = sympify('sf_L_15_R_');
    xOHV    = sympify('sf_L_16_R_');
    xOV     = sympify('sf_L_17_R_');   

    #     Ideal mixing parameters
    idm[0] =   xvA*xMgM13**3*xMgM2**2*xCaM4**2*xSiT1*xOHV**2;
    idm[1] = 2*xvA*xMgM13**3*xAlM2**2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOHV**2;
    idm[2] = 8*xNaA*xMgM13**3*xMgM2*xAlM2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOHV**2;
    idm[3] =   xvA*xMgM13**3*xAlM2**2*xNaM4**2*xSiT1*xOHV**2;
    idm[4] =   xvA*xMgM13**3*xMgM2**2*xMgM4**2*xSiT1*xOHV**2;
    idm[5] =   xvA*xFeM13**3*xFeM2**2*xFeM4**2*xSiT1*xOHV**2;
    idm[6] =   xvA*xMgM13**3*xFeM2**2*xFeM4**2*xSiT1*xOHV**2;
    idm[7] =   xvA*xFeM13**3*xMgM2**2*xFeM4**2*xSiT1*xOHV**2;
    idm[8] =   xvA*xMgM13**3*xFe3M2**2*xNaM4**2*xSiT1*xOHV**2;
    idm[9] = 8*xKA*xMgM13**3*xMgM2*xAlM2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOHV**2;
    idm[10] = 2*xvA*xMgM13**3*xTiM2**2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOV**2;

    
    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');
    gbase[3]  = sympify('gb4');
    gbase[4]  = sympify('gb5');
    gbase[5]  = sympify('gb6');
    gbase[6]  = sympify('gb7');
    gbase[7]  = sympify('gb8');
    gbase[8]  = sympify('gb9');
    gbase[9]  = sympify('gb10');
    gbase[10] = sympify('gb11');
   
    symb     = ['x','y','z','a','k','c','f','t','Q1','Q2']
    sym_list = symb
    
    in_var  = ['x','y','z','a','k','c','f','t','Q1','Q2','log']  
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','x[7]','x[8]','x[9]','ln']
    
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def ilm():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 3

    W         = [0]*n_em
    gbase     = [0]*n_em;
    p         = ['']*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [-0.99,0.99]; # Q range
  
    p[0] = sympify('q');
    p[1] = sympify('x - q');
    p[2] = sympify('1 - x');
    
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 15.6;
    W[1] = 26.6;
    W[2] = 11.0;    
    v = 0.; 

    #     Site fractions
    xFe2A = sympify('1/2*x + 1/2*q');
    xTiA  = sympify('1/2*x - 1/2*q');
    xFe3A = sympify('1 - x');
    xFe2B = sympify('1/2*x - 1/2*q');
    xTiB  = sympify('1/2*x + 1/2*q');
    xFe3B = sympify('1 - x');

    sf    = [xFe2A, xTiA, xFe3A, xFe2B, xTiB, xFe3B];

    xFe2A = sympify('sf_L_0_R_');
    xTiA  = sympify('sf_L_1_R_');
    xFe3A = sympify('sf_L_2_R_');
    xFe2B = sympify('sf_L_3_R_');
    xTiB  = sympify('sf_L_4_R_');
    xFe3B = sympify('sf_L_5_R_'); 
 
    #     Ideal mixing parameters
    idm[0] =   xFe2A*xTiB;
    idm[1] =   4.*xFe2A**(0.5)*xTiA**(0.5)*xFe2B**(0.5)*xTiB**(0.5);
    idm[2] =   xFe3A*xFe3B;

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');

    symb     = ['x','q']
    sym_list = symb
    
    in_var  = ['x','q','log']  
    out_var = ['x[0]','x[1]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def ilm_TJBH_2019():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 3

    W         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [-0.99,0.99]; # Q range
  
    p[0] = sympify('q');
    p[1] = sympify('i - q');
    p[2] = sympify('1 - i');
    
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 7.05;
    W[1] = 14.3;
    W[2] = 7.25;    
    v = 0.; 

    #     Site fractions
    xFeA  = sympify('1/2*i + 1/2*q');
    xTiA  = sympify('1/2*i - 1/2*q');
    xFe3A = sympify('1 - i');
    xFeB  = sympify('1/2*i - 1/2*q');
    xTiB  = sympify('1/2*i + 1/2*q');
    xFe3B = sympify('1 - i');

    sf    = [xFeA, xTiA, xFe3A, xFeB, xTiB, xFe3B];

    xFeA  = sympify('sf_L_0_R_');
    xTiA  = sympify('sf_L_1_R_');
    xFe3A = sympify('sf_L_2_R_');
    xFeB  = sympify('sf_L_3_R_');
    xTiB  = sympify('sf_L_4_R_');
    xFe3B = sympify('sf_L_5_R_'); 
 
    #     Ideal mixing parameters
    idm[0] =   xFeA**(0.5)*xTiB**(0.5);
    idm[1] =   2.*xFeA**(0.25)*xTiA**(0.25)*xFeB**(0.25)*xTiB**(0.25);
    idm[2] =   xFe3A**(0.5)*xFe3B**(0.5);

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');

    symb     = ['i','q']
    sym_list = symb
    
    in_var  = ['i','q','log']  
    out_var = ['x[0]','x[1]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

def spn():
    
    P = symbols('P')
    T = symbols('T')
    
    n_em = 8

    W         = [0]*28
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    idm2      = [0]*n_em;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];  # x  range
    Range[1]  = [0,2.]; # y  range
    Range[2]  = [0,1];  # c  range         # Range(3,:) = [ 0 0.5], Boris correction commented
    Range[3]  = [-1,1]; # t  range
    Range[4]  = [0,1];  # Q1 range
    Range[5]  = [0,1];  # Q2 range
    Range[6]  = [0,1];  # Q3 range

    p[0] = sympify('1/3 - 1/3*x - c + 2/3*Q1 - 2/3*t + (-1/3*t)*x');
    p[1] = sympify('2/3 - 1/3*t - 2/3*Q1 - 2/3*x + (-2/3*t)*x');
    p[2] = sympify('1/3*x - 1/3*y + 2/3*Q2 + 2/3*Q3 + 1/3*t*x + 1/3*c*y + 1/3*t*y');
    p[3] = sympify('-2/3*Q2 - 2/3*Q3 + 2/3*x - 2/3*y + 2/3*t*x + 2/3*c*y + 2/3*t*y');
    p[4] = sympify('1/3*y - 2/3*Q3 + (-1/3*c)*y + (-1/3*t)*y');
    p[5] = sympify('2/3*Q3 + 2/3*y + (-2/3*c)*y + (-2/3*t)*y');
    p[6] = sympify('c');
    p[7] = sympify('t');

    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = -8.2;
    W[1] = 3.5;
    W[2] = -13.0;
    W[3] = 43.2;
    W[4] = 49.1;
    W[5] = -5.0;
    W[6] = 22.5;
    W[7] = 4.4;
    W[8] = -6.0;
    W[9] = 36.8;
    W[10] = 20.0;
    W[11] = 14.0;
    W[12] = 21.5;
    W[13] = -8.2;
    W[14] = 18.1;
    W[15] = 49.0;
    W[16] = -19.0;
    W[17] = 35.1;
    W[18] = -4.0;
    W[19] = 7.6;
    W[20] = -11.0;
    W[21] = 9.0;
    W[22] = 18.1;
    W[23] = 11.9;
    W[24] = 62.2;
    W[25] = -6.4;
    W[26] = 24.3;
    W[27] = 60.0;
    v = 0.;

    #     Site fractions
    xMgT  = sympify('1/3 + 1/3*t - 1/3*x + 2/3*Q1 + (-1/3*t)*x');
    xFeT  = sympify('1/3*x + 2/3*Q2 + 1/3*t*x');
    xAlT  = sympify('2/3 - 1/3*t - 2/3*Q1 - 2/3*Q2 - 2/3*Q3 - 2/3*y + 2/3*c*y + 2/3*t*y');
    xFe3T = sympify('2/3*Q3 + 2/3*y + (-2/3*c)*y + (-2/3*t)*y');
    xMgM  = sympify('1/3 - 1/3*Q1 + 1/3*t - 1/3*x + (-1/3*t)*x');
    xFeM  = sympify('-1/3*Q2 + 1/3*x + 1/3*t*x');
    xAlM  = sympify('2/3 + 1/3*Q1 + 1/3*Q2 + 1/3*Q3 - c - 2/3*y - 5/6*t + 2/3*c*y + 2/3*t*y');
    xFe3M = sympify('-1/3*Q3 + 2/3*y + (-2/3*c)*y + (-2/3*t)*y');
    xCrM  = sympify('c');
    xTiM  = sympify('1/2*t');

    sf   = [xMgT, xFeT, xAlT, xFe3T, xMgM, xFeM, xAlM, xFe3M, xCrM, xTiM];

    xMgT  = sympify('sf_L_0_R_');
    xFeT  = sympify('sf_L_1_R_');
    xAlT  = sympify('sf_L_2_R_');
    xFe3T = sympify('sf_L_3_R_');
    xMgM  = sympify('sf_L_4_R_');
    xFeM  = sympify('sf_L_5_R_');
    xAlM  = sympify('sf_L_6_R_');
    xFe3M = sympify('sf_L_7_R_');
    xCrM  = sympify('sf_L_8_R_');
    xTiM  = sympify('sf_L_9_R_');
    p1    = sympify('sf_L_10_R_');
    
    idm[0] =   xMgT*xAlM;
    idm[1] = 2*xAlT*xMgM**(.5)*xAlM**(.5);
    idm[2] =   xFeT*xAlM;
    idm[3] = 2*xAlT*xFeM**(.5)*xAlM**(.5);
    idm[4] =   xFeT*xFe3M;
    idm[5] = 2*xFe3T*xFeM**(.5)*xFe3M**(.5);
    idm[6] =   xMgT*xCrM;
    idm[7] = 2*xMgT*xMgM**(.5)*xTiM**(.5);
    
    
    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb1');
    gbase[1]  = sympify('gb2');
    gbase[2]  = sympify('gb3');
    gbase[3]  = sympify('gb4');
    gbase[4]  = sympify('gb5');
    gbase[5]  = sympify('gb6');
    gbase[6]  = sympify('gb7');
    gbase[7]  = sympify('gb8');


#     x, y, c, t, Q1, Q2, Q = sympy.symbols('x y c t Q1 Q2 Q3', positive = True, real = True)
    symb     = ['x','y','c','t','Q1','Q2','Q3']
    sym_list = symb
    
    in_var  = ['x','y','c','t','Q1','Q2','Q3','log']  
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','ln']
    
    return sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var

In [ ]:
# CHOOSE PHASE
# bi g ep pl4T mu ol cd opx cpx spn hb liq fl ilm ilm_TJBH_2019
ss = 'spn'
# output
 
name = ss+'()'
sym_list, p, v, W, gbase, sf, idm, idm2, n_em, symmetry, Range, in_var, out_var  = eval(name)

calc_hes = 0;

In [ ]:
# CALCULATE f0, jac and Hessian
R   = sympify("R");
T   = sympify("T");
eps = sympify("eps");

phi_p           = np.identity(n_em);
mu_Gex          = [0]*n_em;
mat_muMix       = [0]*n_em;
mu_Total        = [0]*n_em;
mat_phi         = [0]*n_em;
Gsol            = 0;
total_chem_comp = 0;

#declare mu0[ii] and p[ii] again here
muzx  = [0]*n_em
px    = [0]*n_em
for ii in range(0,n_em):
    px[ii]    = sympify('p'+str(ii))
    muzx[ii]  = sympify('muz'+str(ii))

if symmetry == 0:
    # Third step
    dp_dx = [[0] * (n_em-1) for i in range(n_em)]
    for ii in range(0,n_em):
        for jj in range(0,n_em-1):
            dp_dx[ii][jj] = diff(p[ii],sym_list[jj]) 
            
    # Second step            
    pfrac = p        
    p  = [0]*n_em
    px = [0]*n_em
    for ii in range(0,n_em):
        p[ii]  = sympify('p'+str(ii))
        px[ii] = sympify('p'+str(ii))
        
    sum_v   = sum(np.multiply(p,v)) 
    mat_phi = np.multiply(p,v)/(sum_v)
    mat_phix= mat_phi

    mat_phi = [0]*n_em
    for ii in range(0,n_em):
        mat_phi[ii] = sympify('mat_phi'+str(ii))

    for ii in range(0,n_em):
        mu = 0;
        it = 0;
        for jj in range (0,n_em-1):
            for kk in range(jj+1,n_em):
                mu = mu - (phi_p[ii][jj] - mat_phi[jj])*(phi_p[ii][kk]-mat_phi[kk])*(W[it]*2*v[ii]/(v[jj]+v[kk]) );
                it = it + 1;
        if len(W) != it:
            display("Something is wrong with the mu model")
            
        mu_Gex[ii]      = mu; 
        Gsol            = Gsol + px[ii]*muzx[ii];

if symmetry == 1:
    mat_phix = []
    
    # Third step
    dp_dx = [[0] * (n_em-1) for i in range(n_em)]
    for ii in range(0,n_em):
        for jj in range(0,n_em-1):
            dp_dx[ii][jj] = diff(p[ii],sym_list[jj]) 

    pfrac = p       
    p  = [0]*n_em
    px = [0]*n_em
    
    for ii in range(0,n_em):
        p[ii]  = sympify('p'+str(ii))
        px[ii] = sympify('p'+str(ii))
        
    for ii in range(0,n_em):
        mu = 0;
        it = 0;
        for jj in range (0,n_em-1):
            for kk in range(jj+1,n_em):
                mu = mu - (phi_p[ii][jj] - p[jj])*(phi_p[ii][kk]-p[kk])*W[it];
                it = it + 1;
        if len(W) != it:
            display("Something is wrong with the mu model")
            
        mu_Gex[ii]      = mu; 
        Gsol            = Gsol + px[ii]*muzx[ii];
        
mx = [0]*n_em
for ii in range(0,n_em):
    mx[ii] = sympify('idm'+str(ii))
     
mu_Gexx = [0]*n_em        
for ii in range(0,n_em):
    mu_Gexx[ii] = sympify('mu_Gex'+str(ii))
        
muz = [0]*n_em
for ii in range(0,n_em):
    muz[ii] = (gbase[ii] + mu_Gexx[ii] + R*T*ln(mx[ii]))

f0 = 0        
for ii in range(0,len(gbase)):
    f0 = f0 + (sympify('muz'+str(ii)))*px[ii]

# Site fraction      
sf_size   = len(sf)
sf0       = [0]*sf_size

for oo,ii in enumerate(sf):
    sf0[oo] = -1.0*sf[oo] 

sym_list_size = len(sym_list)
ineq_sf = [['' for xx in range(sym_list_size)] for xx in range(sf_size)]

for oo,ii in enumerate(sf):
    for pi,jj in enumerate(sym_list):
        ineq_sf[oo][pi]  = diff(ii,jj);  
        ineq_sf[oo][pi] *= -1.0;

# IDM ineq
idm_size   = len(idm)
sym_list_size = len(sym_list)
ineq_idm = [['' for xx in range(sym_list_size)] for xx in range(idm_size)]

for oo,ii in enumerate(idm):
    for pi,jj in enumerate(sym_list):
        ineq_idm[oo][pi]= diff(ii,jj)*1.0;
        
# turn rational and int to float
for i,b in enumerate(idm):
    for a in preorder_traversal(b):
        if isinstance(a, Rational):
            idm[i] = idm[i].subs(a, float(a))
            
for i,b in enumerate(pfrac):
    for a in preorder_traversal(b):
        if isinstance(a, Rational):
            pfrac[i] = pfrac[i].subs(a, float(a))

# for i,b in enumerate(dp_dx):
#     for j,c in enumerate(b):
#         for a in preorder_traversal(c):
#             if isinstance(a, Rational):
#                 dp_dx[i][j] = dp_dx[i][j].subs(a, float(a))

for i,b in enumerate(sf):
    for a in preorder_traversal(b):
        if isinstance(a, Rational):
            sf[i] = sf[i].subs(a, float(a))
            
for i,b in enumerate(sf0):
    for a in preorder_traversal(b):
        if isinstance(a, Rational):
            sf0[i] = sf0[i].subs(a, float(a)) 
            
for i,b in enumerate(ineq_sf):
    for j,c in enumerate(b):
        for a in preorder_traversal(c):
            if isinstance(a, Rational):
                ineq_sf[i][j] = ineq_sf[i][j].subs(a, float(a))
                
for ii, val in enumerate(ineq_sf):
    for jj, val0 in enumerate(val):
        t0 = str(val0)
        for i,val in enumerate(in_var):
            inv  = str(in_var[i])
            outv = str(out_var[i])
            t0 = replaceSymbols(t0,inv,outv) 

        ineq_sf[ii][jj] = t0

for ii, val in enumerate(sf0):  
    t0 = str(val)

    for i,val in enumerate(in_var):
        inv  = str(in_var[i])
        outv = str(out_var[i])
        t0 = replaceSymbols(t0,inv,outv) 
    sf0[ii] = t0    
    
idm_p  = [0]*n_em
for jj in range(0,n_em):
    idm_p[jj] = -1.0*(idm[jj]/pfrac[jj])
    
for ii, val in enumerate(idm_p):  
    t0 = str(val)

    for i,val in enumerate(in_var):
        inv  = str(in_var[i])
        outv = str(out_var[i])
        t0 = replaceSymbols(t0,inv,outv) 
    idm_p[ii] = str(t0)

In [ ]:
#print vectorized non linear constraints (sf)
if 0==0:
        
    op = ''
    os = '    '
    
    op += 'void '+ss+'_c(unsigned m, double *result, unsigned n, const double *x, double *grad, void *data){\n'
    for ii, val0 in enumerate(ineq_sf):
#         op += os+'double c'+ str(ii)+'(unsigned n, const double *x, double *grad, void *data){\n'   
        op += os+'result['+str(ii)+'] = ( eps_sf + ' + str(sf0[ii])+');\n' 
    op += '\n' 
    op += os+'if (grad) {\n' 
    n = 0;
    for ii, val0 in enumerate(ineq_sf):
        for jj, val1 in enumerate(val0):
            op += os+os+'grad['+str(n)+'] = ' + str((val1)) + ';\n' 
            n += 1
    op += os+'}\n' 
    
    op += '\n'  
    op += os+'return;\n'    
    op += '};\n' 
    op += '\n'  
    
print(op)

In [ ]:
# site fractions
if 0==0:
    pout = sf

    for ii, val in enumerate(sf):
        t0 = str(val)
        for i,val in enumerate(in_var):
            inv  = str(in_var[i])
            outv = str(out_var[i])
            t0 = replaceSymbols(t0,inv,outv) 

        pout[ii] = t0


    op = ''
    os = '    '

    for  i,val in enumerate(pout):
        op += os+'sf['+str(i)+']           = '+((pout[i]))+';\n'    

print(op)


In [ ]:
# dp dx
if 0==0:

    pout = (dp_dx)
    for ii, val in enumerate(dp_dx):
        for jj, val0 in enumerate(val):
            t0 = (str((val0)))
            for i,val in enumerate(in_var):
                inv  = str(in_var[i])
                outv = str(out_var[i])
                t0 = replaceSymbols(t0,inv,outv) 

            pout[ii][jj] = t0


    op = ''
    os = '    '
    op += os+'\n' 
    op += 'void dpdx_'+ss+'(void *SS_ref_db, const double *x){\n'
    op += os+'SS_ref *d  = (SS_ref *) SS_ref_db;\n'
    op += os+'double **dp_dx = d->dp_dx;\n\n'

    for  i,val in enumerate(pout):
        for j,val2 in enumerate(val):
            op += os+'dp_dx['+str(i)+']['+str(j)+'] = '+((pout[i][j]))+';  '  
        op += os+'\n'
    op += '}\n'
    
print(op)

In [ ]:
# endmember fractions (p)
if 0==0:
    pout = p

    for ii, val in enumerate(pfrac):
        t0 = str(val)
        for i,val in enumerate(in_var):
            inv  = str(in_var[i])
            outv = str(out_var[i])
            t0 = replaceSymbols(t0,inv,outv) 

        pout[ii] = t0


    op = ''
    os = '    '
    op += os+'\n' 
    op += 'void px_'+ss+'(void *SS_ref_db, const double *x){\n'
    op += os+'SS_ref *d  = (SS_ref *) SS_ref_db;\n'
    op += os+'double *p = d->p;;\n'
    
    for  i,val in enumerate(pfrac):
        op += os+'    p['+str(i)+']           = '+((pout[i]))+';\n'    
    op += '}\n'
    
print(op)

In [ ]:
# other objective analytical formulation
if 0==0:
    # PRINT OUT THE RESULTS FOR PYTHON
    op = ''
    os = '    '
    op += os+'\n' 
    op += 'SS_ref XI_SS_'+ss+'_function(SS_ref SS_ref_db, double *bulk_rock, double P,  double T, double obj_res, double ineq_res, int solver, double eps_bnds){\n' 
    op += os+'\n' 
    op += os+'/* OBJECTIVE FUNCTION AND GRADIENT DEFINITION */\n'
    op += os+'double obj(unsigned n, const double *x, double *grad, void *SS_ref_db) {\n'  
    op += os+'\n'  
    op += os+'    SS_ref *d  = (SS_ref *) SS_ref_db;\n\n'
    op += os+'    int n_em   = d->n_em;\n'
    op += os+'    double P   = d->P;\n'
    op += os+'    double T   = d->T;\n'
    op += os+'    double R   = d->R;\n\n'

    for ii in range(0,len(sym_list)):
        sym = str(sym_list[ii])
        if sym == 'x':
            sym += '1'
        if sym == 'n':
            sym += '1'
        op += os+'    double '+sym+' = x['+str(ii)+'];\n'
    op += os+'\n'

    op += os+'    double *gb     = d->gb_lvl;\n'  
    op += os+'    double *p      = d->p;\n'
    op += os+'    double *mat_phi= d->mat_phi;\n'
    op += os+'    double *mu_Gex = d->mu_Gex;\n'
    op += os+'    double *idm    = d->idm;\n'
    op += os+'    double *sf     = d->sf;\n'
    op += os+'    double *mu     = d->mu;\n'
    op += os+'    double *dfx    = d->dfx;\n'
    op += os+'    double **dp_dx = d->dp_dx;\n\n'

    for  i,val in enumerate(pfrac):
        op += os+'    p['+str(i)+']           = '+digit2index((pfrac[i]))+';\n'    
    op += os+'\n'

    if any(mat_phix):
        for  i,val in enumerate(mat_phix):
            op += os+'    mat_phi['+str(i)+']    = '+digit2index((mat_phix[i]))+';\n'    
        op += os+'\n'   

    for  i,val in enumerate(mu_Gex):
        op += os+'    mu_Gex['+str(i)+']      = '+digit2index((mu_Gex[i]))+';\n'    
    op += os+'\n'

#     for  i,val in enumerate(sf):
#         op += os+'    sf['+str(i)+']          = '+digit2index(ccode(sf[i]))+';\n'    
#     op += os+'\n'

    for  i,val in enumerate(idm):
        op += os+'    idm['+str(i)+']         = '+brackets(ccode(digit2index((idm[i]))))+';\n'    
    op += os+'\n'

    for  i,val in enumerate(muz):
        op += os+'    mu['+str(i)+']          = '+digit2index2((muz[i]))+';\n'    
    op += os+'\n'

    for  i,val in enumerate(dp_dx):
        for j,val2 in enumerate(val):
            op += os+'    dp_dx['+str(i)+']['+str(j)+'] = '+digit2index((dp_dx[i][j]))+';  '  
        op += os+'\n'
    op += os+'\n'

    op += os+'    for (int i = 0; i < (n_em-1); i++){\n'
    op += os+'        dfx[i] = 0.0;\n'
    op += os+'        for (int j = 0; j < n_em; j++){\n'
    op += os+'            dfx[i] += mu[j]*dp_dx[j][i];\n'    
    op += os+'        }\n' 
    op += os+'    }\n\n' 

    op += os+'    if (grad) {\n'   
    for ii in range(0,len(sym_list)):
        op += os+'        grad['+str(ii)+'] = creal(dfx['+str(ii)+']);\n'
    op += os+'    }\n'
    op += os+'    return ('+digit2index(str((f0)))+');\n'
    op += os+'};\n\n'

    op += os+'\n'
    op += os+'/* INEQUALITY CONSTRAINTS AND JAC */\n'

    for ii, val0 in enumerate(ineq_sf):
        op += os+'double c'+ str(ii)+'(unsigned n, const double *x, double *grad, void *data){\n'      
        op += os+'   if (grad) {\n'    

        for jj, val1 in enumerate(val0):
            op += os+'        grad['+str(jj)+'] = ' + str((val1)) + ';\n' 
        op += os+'   }\n' 
        op += os+'    return (' + str(sf0[ii])+');\n' 
        op += os+'};\n' 

    op += os+'\n'  
    op += os+'int    n_em     = SS_ref_db.n_em;\n'  
    op += os+'unsigned int n  = n_em -1;\n\n'  

    op += os+'double *x  = SS_ref_db.iguess; \n'  
    op += os+'double lb[n_em-1]; \n'  
    op += os+'double ub[n_em-1]; \n\n'  
    op += os+'for (int i = 0; i < (n_em-1); i++){\n'  
    op += os+'   lb[i] = SS_ref_db.box_bounds[i][0] + eps_bnds;\n'  
    op += os+'   ub[i] = SS_ref_db.box_bounds[i][1] - eps_bnds;\n' 
    op += os+'}\n'  

    op += os+'\n'    
    op += os+'/* DEFINE NLOPT SOLVER as LD_ MMA, SLSQP, CCSAQ */\n'
    op += os+'clock_t t; \n'  
    op += os+'t = clock();\n'  
    op += os+'nlopt_opt opt;\n'  
    op += os+'\n'  
    op += os+'if (solver == 0){\n'  
    op += os+'    opt = nlopt_create(NLOPT_LD_CCSAQ, (n));\n'  
    op += os+'}\n'  
    op += os+'else if (solver == 1){\n' 
    op += os+'    opt = nlopt_create(NLOPT_LD_MMA, (n));\n'  
    op += os+'}\n'  
    op += os+'else if (solver == 2){\n' 
    op += os+'    opt = nlopt_create(NLOPT_LD_SLSQP, (n));\n'  
    op += os+'}\n'
    op += os+'nlopt_set_lower_bounds(opt, lb);\n'  
    op += os+'nlopt_set_upper_bounds(opt, ub);\n'  
    op += os+'nlopt_set_min_objective(opt, obj, &SS_ref_db);\n'  

    for ii in range(0,len(sf)):
        sym = str(ii)
        op += os+'nlopt_add_inequality_constraint(opt, c'+sym+', NULL, ineq_res);\n'
    op += os+'nlopt_set_ftol_rel(opt, obj_res);\n' 
    op += os+'\n' 
    op += os+'nlopt_set_maxtime(opt, maxtime);\n' 
    op += os+'\n' 
    op += os+'double minf;\n' 
    op += os+'if (nlopt_optimize(opt, x, &minf) < 0) {\n' 
    op += os+'    printf("  -> nlopt failed!\\n");\n' 
    op += os+'}\n' 
    op += os+'\n' 
    op += os+'t = clock() - t; \n' 
    op += os+'SS_ref_db.LM_time = ((double)t)/CLOCKS_PER_SEC*1000; // in seconds \n' 
    op += os+'\n' 
    op += os+'nlopt_destroy(opt);\n' 
    op += os+'\n' 
    op += os+'/* Send back needed local solution parameters */\n' 
    op += os+'SS_ref_db.xeos = x; \n' 
    op += os+'SS_ref_db.df   = minf; \n' 
    op += os+'\n' 
    op += os+'return SS_ref_db;\n' 
    op += os+'\n'
    op += '};\n'    




    print(op) 

In [ ]:
# GENERATE JACOBIAN MATRIX OF SITE FRACTIONS #
sp  = '\t'

pp  = '/**\n'
pp += 'update dsf matrix: ' + ss + '\n'
pp += '*/\n'
pp += 'void dsf_'+ss+'(void *SS_ref_db, const double *x){\n'
pp += sp+'SS_ref *d  = (SS_ref *) SS_ref_db;\n'
pp += sp+'double **dsf = d->dsf;\n'
pp += '\n'
for ii, val0 in enumerate(ineq_sf): 
    pp += sp
    for jj, val1 in enumerate(val0):
        pp += 'dsf['+str(ii)+']['+str(jj)+']\t= ' + str((val1)) + '; ' 
    pp += '\n'  
pp += '};\n\n'


print(pp)

In [6]:
print(ol)

[[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
